<a href="https://colab.research.google.com/github/cboyda/LighthouseLabs/blob/main/Project_Stats_city_bikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CityBikes

Send a request to CityBikes for the city of your choice. Ideas gained from https://medium.com/@ajosegun_/real-time-dashboard-in-python-b8c9a9c4e050

In [3]:
import requests
import pandas as pd
import plotly.express as px
import datetime

In [4]:
city_bikes = requests.get("http://api.citybik.es/v2/networks").json()

In [5]:
# city_bikes # list all api returns
# {'networks': [{'company': ['ЗАО «СитиБайк»'],
#    'href': '/v2/networks/velobike-moscow',
#    'id': 'velobike-moscow',
#    'location': {'city': 'Moscow',
#     'country': 'RU',
#     'latitude': 55.75,
#     'longitude': 37.616667},
#    'name': 'Velobike'},
#   {'company': ['Urban Infrastructure Partner'],
#    'href': '/v2/networks/baerum-bysykkel',
#    'id': 'baerum-bysykkel',
#    'location': {'city': 'Bærum',
#     'country': 'NO',
#     'latitude': 59.89455,
#     'longitude': 10.546343},
#    'name': 'Bysykkel'},...

In [6]:
cities = set()  # Using a set to store unique cities

for network in city_bikes['networks']:
    city = network['location']['city']
    country = network['location']['country']
    cities.add((city, country))  # Store city and country as a tuple

unique_cities = list(cities)  # Convert the set back to a list

unique_cities.sort()
# unique_cities


In [7]:
for city in unique_cities:
  if city[1] == 'CA':
    print(city)

('Hamilton, ON', 'CA')
('Montréal, QC', 'CA')
('Québec', 'CA')
('Saguenay', 'CA')
('Toronto, ON', 'CA')
('Vancouver', 'CA')


In [ ]:
# select a city
city = "Vancouver"

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [8]:
def get_city_data(city):
  city_bikes = requests.get("http://api.citybik.es/v2/networks").json()

  list_of_dicts = []
  for city_bike_dict in city_bikes['networks']:
    new_city = city_bike_dict['location']['city']
    if new_city.lower() == city.lower():
      list_of_dicts.append(city_bike_dict)
  return list_of_dicts

In [9]:
get_city_data(city)

[{'company': ['Vanncouver Bike Share Inc.',
   'CycleHop LLC',
   'City of Vancouver',
   'Shaw Communications Inc.',
   'Fifteen'],
  'gbfs_href': 'https://vancouver-gbfs.smoove.pro/gbfs/2/gbfs.json',
  'href': '/v2/networks/mobibikes',
  'id': 'mobibikes',
  'location': {'city': 'Vancouver',
   'country': 'CA',
   'latitude': 49.2827,
   'longitude': -123.1207},
  'name': 'Mobi'}]

In [10]:
def get_stations_info(city):
  station_dict = get_city_data(city)
  if not station_dict:
    print("Error: No bike company found for {}",format(city))
    return None

  network_address=station_dict[0]['href']
  url="http://api.citybik.es/{}".format(network_address)
  return requests.get(url).json()['network']['stations']
# The full network URL here is http://api.citybik.es/v2/networks/velib

In [11]:
station_info = get_stations_info(city)
#station_info

# SAMPLE RESULTS
# [{'empty_slots': 32,
#   'extra': {'ebikes': 0,
#    'has_ebikes': True,
#    'last_updated': 1685669893,
#    'normal_bikes': 3,
#    'renting': True,
#    'returning': True,
#    'slots': 35,
#    'uid': '0001'},
#   'free_bikes': 3,
#   'id': '7a19c49f486d7c0c02b3685d7b240448',
#   'latitude': 49.262487,
#   'longitude': -123.114397,
#   'name': '10th & Cambie',
#   'timestamp': '2023-06-02T01:43:36.724000Z'},
#  {'empty_slots': 11,
#   'extra': {'ebikes': 2,
#    'has_ebikes': True,
#    'last_updated': 1685669744,
#    'normal_bikes': 3,
#    'renting': True,
#    'returning': True,
#    'slots': 16,
#    'uid': '0004'},
#   'free_bikes': 5,
#   'id': '32603a87cfca71d0f7dfa3513bad69d5',
#   'latitude': 49.274566,
#   'longitude': -123.121817,
#   'name': 'Yaletown-Roundhouse Station',
#   'timestamp': '2023-06-02T01:43:36.722000Z'}]

Put your parsed results into a DataFrame.

In [12]:
def get_available_stations(city):
  '''
  Takes in the city name and returns a pandas dataframe containing information about the city
  '''
  station_info = get_stations_info(city)

  station_list = []
  for info in station_info:
    a_dict = {
        'Station Name': info['name'],
        'empty_slots': info['empty_slots'],
        'slots': info['extra']['slots'],
        'free_bikes': info['free_bikes'],
        'ebikes': info['extra']['ebikes'],
        #'normal_bikes': info['extra']['normal_bikes'],
        #'payment': ','.join(info['extra']['payment']) if info['extra']['banking'] else "No",
        'latitude': info['latitude'],
        'longitude': info['longitude'],
        'timestamp': info['timestamp'],
        'Unique ID': info['extra']['uid'],
    }
    station_list.append(a_dict)
  return pd.DataFrame(station_list)

In [13]:
df = get_available_stations(city)

In [14]:
df

,Station Name,empty_slots,slots,free_bikes,ebikes,latitude,longitude,timestamp,Unique ID
0,10th & Cambie,32,35,3,0,49.262487,-123.114397,2023-06-02T02:09:24.467000Z,0001
1,Yaletown-Roundhouse Station,11,16,5,2,49.274566,-123.121817,2023-06-02T02:09:24.463000Z,0004
2,Dunsmuir & Beatty,25,26,1,0,49.279764,-123.110154,2023-06-02T02:09:24.464000Z,0005
3,12th & Yukon (City Hall),13,16,3,3,49.260599,-123.113504,2023-06-02T02:09:24.466000Z,0007
4,8th & Ash,16,16,0,0,49.264215,-123.117772,2023-06-02T02:09:24.475000Z,0008
...,...,...,...,...,...,...,...,...,...
237,Burrard & 14th,11,18,6,5,49.259469,-123.145718,2023-06-02T02:09:24.348000Z,0210
238,Hornby & Drake,22,24,1,0,49.277178,-123.130000,2023-06-02T02:09:24.365000Z,0196
239,Cardero & Bayshore,3,20,15,4,49.291597,-123.129158,2023-06-02T02:09:24.308000Z,0200
240,27th & Main,12,22,10,6,49.247204,-123.101549,2023-06-02T02:09:24.532000Z,0565


Show information map.

In [34]:
# def show_map(station_data):
#   '''
#   Takes in data of the station info in a dataframe format then
#   Shows it on a map
#   '''
#   # get the current data and time
#   current_date = pd.to_datetime(station_data['timestamp'][0]).strftime('%a %d %B %Y at %H:%M')

#   map_title = 'Map Showing Number of Bikes in {} at {}'.format(city, current_date)

#   fig = px.scatter_mapbox(station_data, lat="latitude",lon="longitude", hover_name="Station Name", color="free_bikes",
#                           hover_data=["empty_slots", "free_bikes", "ebikes"],
#                           title=map_title,
#                           color_continuous_scale=px.colors.sequential.Plasma,
#                           size_max=20,
#                           zoom=12)
#   fig.show(renderer="colab")

In [47]:
# show map rewrite to render in Google Colab
import folium

def show_map(live_data):
    '''
    Takes in the live data Pandas dataframe to show a map of the stations,
    return nothing.
    '''

    if live_data.empty:
        print("Error: Station data is not available!")
        return None

    # Create a map centered around the latitude and longitude of the first station
    map = folium.Map(location=[live_data['latitude'].iloc[0], live_data['longitude'].iloc[0]], zoom_start=13)

    # Add markers for each station
    for index, row in live_data.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=row['Station Name'],
            icon=folium.Icon(color='blue')
        ).add_to(map)

    # Display the map
    display(map)


In [16]:
#! pip install plotly --upgrade # my plotly.express map was not showing now at 5.14.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!pip show plotly

Name: plotly
Version: 5.14.1
Summary: An open-source, interactive data visualization library for Python
Home-page: https://plotly.com/python/
Author: Chris P
Author-email: chris@plot.ly
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: packaging, tenacity
Required-by: cufflinks, datascience, plotly-express


In [22]:
# confirm latest version installed as I was having display problems of the maps/plots
import plotly
plotly.__version__

'5.14.1'

In [49]:
live_station_data = get_available_stations("Vancouver")
show_map(live_station_data)

In [44]:
# v2 but no display
# def show_pie_chart(live_data, station_name):
#     '''
#     Takes in the live data Pandas dataframe and requested station name to
#     show a pie chart, return nothing.
#     '''

#     if live_data.empty:
#         print("Error: Station data is not available!")
#         return None

#     requested_station_data = live_data[live_data["Station Name"].map(lambda name: station_name.lower() in name.lower())]

#     if requested_station_data.empty:
#         print("Error: Requested station not found!")
#         return None

#     current_date = pd.to_datetime(requested_station_data['timestamp'].iloc[0]).strftime('%a %d %B %Y at %H:%M')

#     title = "Bike information for: {0} at {1}".format(
#         requested_station_data["Station Name"].to_string(index=False),
#         current_date)

#     fig = px.pie(requested_station_data, values=["empty_slots", "free_bikes", "ebikes"],
#                  names=["Empty Slots", "Free Bikes", "ebikes"],
#                  hole=.3,
#                  title=title)

#     fig.update_traces(textposition='inside',
#                       textinfo='value+label')

#     fig.show()


If the pie chart is not being displayed in Google Colab, it could be due to the rendering settings or limitations of the Colab environment. To ensure that the chart is displayed, you can try using the plotly.offline mode with the iplot function.

Here's an updated version of the show_pie_chart function that uses plotly.offline.

In [45]:
import plotly.offline as pyo
import plotly.graph_objs as go

def show_pie_chart(live_data, station_name):
    '''
    Takes in the live data Pandas dataframe and requested station name to
    show a pie chart, return nothing.
    '''

    if live_data.empty:
        print("Error: Station data is not available!")
        return None

    requested_station_data = live_data[live_data["Station Name"].map(lambda name: station_name.lower() in name.lower())]

    if requested_station_data.empty:
        print("Error: Requested station not found!")
        return None

    current_date = pd.to_datetime(requested_station_data['timestamp'].iloc[0]).strftime('%a %d %B %Y at %H:%M')

    title = "Bike information for: {0} at {1}".format(
        requested_station_data["Station Name"].to_string(index=False),
        current_date)

    data = [
        go.Pie(
            labels=["Empty Slots", "Free Bikes", "ebikes"],
            values=[requested_station_data["empty_slots"].iloc[0],
                    requested_station_data["free_bikes"].iloc[0],
                    requested_station_data["ebikes"].iloc[0]],
            hole=0.3,
            textposition='inside',
            textinfo='value+label'
        )
    ]

    layout = go.Layout(title=title)

    fig = go.Figure(data=data, layout=layout)

    pyo.iplot(fig)


In [50]:
live_city_data = get_available_stations(city)
show_pie_chart(live_city_data,city)